In [1]:
import numpy as np
import numpy.testing as npt
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.stats
import logging
from collections import defaultdict

In [2]:
import gym
env = gym.make('FrozenLake-v0' )

[2018-01-05 17:01:17,219] Making new env: FrozenLake-v0


In [3]:
state  = env.reset()

In [4]:
def act(action):
    state, reward, done, info = env.step(action)
    print (state)
    env.render()  

In [5]:
act(3)

0
  (Up)
SFFF
FHFH
FFFH
HFFG


In [6]:
pi = {0:1, 1:2, 2:1, 3:0, 4:1, 6:1, 8:2, 9:0, 10:1, 13:2, 14:2}

In [7]:
state  = env.reset()
print (state)
done = False
while not done:
    env.render()
    action = pi[state]
    print ("random action:", action)
    state, reward, done, info = env.step(action)
    print (state, reward, done, info)
    if done:
        print ("return:", reward) # return for all visited states is here last reward

env.render(close=True)

0

SFFF
FHFH
FFFH
HFFG
random action: 1
0 0.0 False {'prob': 0.3333333333333333}
  (Down)
SFFF
FHFH
FFFH
HFFG
random action: 1
4 0.0 False {'prob': 0.3333333333333333}
  (Down)
SFFF
FHFH
FFFH
HFFG
random action: 1
5 0.0 True {'prob': 0.3333333333333333}
return: 0.0


In [8]:
policy = lambda a: pi[a]

In [9]:
def first_visit_monte_carlo_prediction(mc_policy, mc_env, target_state, nb_episodes):
    returns_sum = defaultdict(float)
    returns_count = defaultdict(float)
    
    V = defaultdict(float)
    
    for i in range(1, nb_episodes + 1):
        episode = []
        state = mc_env.reset()
        done = False
        while not done:
            action = mc_policy(state)
            next_state, reward, done, _ = mc_env.step(action)
            episode.append((state, action, reward))
            if done:
                break
            state = next_state

        # Find all states the we've visited in this episode
        # We convert each state to a tuple so that we can use it as a dict key
        states_in_episode = set([tuple(x) for x in episode])
        print(states_in_episode)
        for state in states_in_episode:
            first_occurence_idx = next(i for i,x in enumerate(episode) if x[0] == state)
            G = sum([x[2]*(discount_factor**i) for i,x in enumerate(episode[first_occurence_idx:])])
            returns_sum[state] += G
            returns_count[state] += 1.0
            V[state] = returns_sum[state] / returns_count[state]

    return V

In [10]:
v_s = first_visit_monte_carlo_prediction(mc_policy=policy, mc_env=env, target_state=0, nb_episodes=10000)
print (v_s)

{(8, 2, 0.0), (0, 1, 0.0), (4, 1, 0.0)}


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()